# ナップサック問題のコードを流用

In [1]:
class KnapsackProblem(object):
    """ The difinition of KnapSackProblem """    
    def __init__(self, name, capacity, items, costs, weights,
                 zeros=set(), ones=set()):
        self.name = name
        self.capacity = capacity
        self.items = items
        self.costs = costs
        self.weights = weights
        self.zeros = zeros # ナップサックに入れない品物の集合
        self.ones = ones # ナップサックに入れる品物の集合
        self.lb = -100
        self.ub = -100
        ratio = {j:costs[j]/weights[j] for j  in items}
        self.sitemList =  [k for k, v in 
            sorted(ratio.items(), key=lambda x:x[1], reverse=True)] # 品物を効率の大きい順に並べたリスト
        self.xlb = {j:0 for j in self.items} # lb を達成する解
        self.xub = {j:0 for j in self.items} # ub を達成する解
        self.bi = None # ubを達成する際に値が分数であるもの

    def getbounds(self):
        """ Calculate the upper and lower bounds. """
        # ナップサックに入れない品物を0に変更
        for j in self.zeros:
            self.xlb[j] = self.xub[j] = 0
        # ナップサックに入れる品物を1に変更
        for j in self.ones:
            self.xlb[j] = self.xub[j] = 1
        # もしナップサックに入れる品物だけでcapacityを超えた場合は計算終了
        if self.capacity < sum(self.weights[j] for j in self.ones):
            self.lb = self.ub =  -100
            return 0
        
        # 決定していない品物のセット
        ritems = self.items - self.zeros - self.ones
        # ritemsを効率の良い順に並べたリスト
        sitems = [j for j in self.sitemList if j in ritems]
        # 残りの容量を計算
        cap = self.capacity - sum(self.weights[j] for j in self.ones)
        # 効率の良い順にループ
        for j in sitems:
            # もし容量に余裕があったら、1に変更
            if self.weights[j] <= cap:
                cap -= self.weights[j]
                self.xlb[j] = self.xub[j] = 1
            # 容量に余裕がない場合は ub は容量分だけ追加
            # lbの計算のために次のループに移動
            else:
                self.xub[j] = cap/self.weights[j]
                self.bi = j # biを更新
                break
        # lb,ubを計算
        self.lb = sum(self.costs[j]*self.xlb[j] for j in self.items)
        self.ub = sum(self.costs[j]*self.xub[j] for j in self.items)

    def __str__(self):
        """ KnapSackProblem の情報を印字 """
        return('Name = '+self.name+', capacity = '+str(self.capacity)+',\n'
            'items = '+str(self.items)+',\n'+
            'costs = '+str(self.costs)+',\n'+
            'weights = '+str(self.weights)+',\n'+
            'zeros = '+str(self.zeros)+', ones = '+str(self.ones)+',\n'+
            'lb = '+str(self.lb)+', ub = '+str(self.ub)+',\n'+
            'sitemList = '+str(self.sitemList)+',\n'+
            'xlb = '+str(self.xlb)+',\n'+'xub = '+str(self.xub)+',\n'+
            'bi = '+str(self.bi)+'\n')

In [2]:
from pulp import *

def KnapsackProblemSolve(capacity, items, costs, weights):
    # dequeを使うとリストよりも計算コストが少なく、先頭と末尾に追加可能
    from collections import deque
    queue = deque()
    
    # 問題を設定する
    root = KnapsackProblem('KP', capacity = capacity,
        items = items, costs = costs, weights = weights,
        zeros = set(), ones = set())
    # 下界、上界を計算する
    root.getbounds()
    best = root
    # 問題を追加する
    queue.append(root)
    
    # queueが空になるまで
    while queue != deque([]):
        # 一番左を選択 
        p = queue.popleft()
        p.getbounds()
        # bestのlb よりも pのubが大きい場合(bestを更新する可能性がある)
        if p.ub > best.lb:
            # もしpのlbがbestのlbよりも大きければ更新
            if p.lb > best.lb:
                best = p
            # pのubがpのlbよりも大きければ子問題を作って分岐する
            if p.ub > p.lb:
                # ubを求める際に分数になった番号をkに格納
                k = p.bi
                # kを1,0に分割し、ququeに追加
                p1 = KnapsackProblem(p.name+'+'+str(k),
                    capacity = p.capacity, items = p.items,
                    costs = p.costs, weights = p.weights,
                    zeros = p.zeros, ones = p.ones.union({k}))
                queue.append(p1)
                p2 = KnapsackProblem(p.name+'-'+str(k),
                    capacity = p.capacity, items = p.items,
                    costs = p.costs, weights = p.weights,
                    zeros = p.zeros.union({k}), ones = p.ones)
                queue.append(p2)
    return 'Optimal', best.lb, best.xlb

# 列生成法

In [3]:
from pulp import *
import numpy as np
MEPS = 1.0e-8

def binpacking(capacity, w):
    m = len(w)
    items = set(range(m))
    A = np.identity(m) # 単位行列

    solved = False
    columns = 0
    
    # D-LBPを定義
    dual = LpProblem(name='D(K)', sense=LpMaximize)
    y = [LpVariable('y'+str(i), lowBound=0) for i in items]

    dual += lpSum(y[i] for i in items) # 目的関数の設定
    for j in range(len(A.T)): # 制約条件の付加(この時点では y1<1, y2<1 が生成される)
        dual += lpDot(A.T[j],y) <= 1, 'ineq'+str(j)

    while not(solved):
        # D-LBPを解く
        dual.solve()
        
        # ナップサック問題を解く
        costs = {i: y[i].varValue for i in items}
        weights = {i: w[i] for i in items}
        print("列生成：{} の計算を行います...".format(columns))
        # ナップサック問題におけるお金がcosts, 重さがweights
        print("costs:", costs)
        print("weights:", weights)
        (state, val, sol) = KnapsackProblemSolve(capacity, items, costs, weights)
        # 最適値がval, その時の解がsol
        print("val: ", val)
        print("sol: ", sol)
       
        # 最適値が1より大きければD-LBPに問題を加えて解きなおす
        if val >= 1.0+MEPS:
            # solを配列にしてaに格納
            a = np.array([int(sol[i]) for i in items])
            # ここで y2 + y3 + y5 + y6 <= 1 などが生成される(solが1の部分に対応)
            dual += lpDot(a, y) <= 1, 'ineq'+str(m+columns)
            A = np.hstack((A, a.reshape((-1,1))))
            columns += 1
            print('A:\n',A)
        # 最適値が1なら解けた
        else:
            solved = True

    print('Generated columns: ', columns)
    m, n = A.shape
    
    # 上記が解けた時のKを使って元の問題を解く
    primal = LpProblem(name='P(K)', sense=LpMinimize)
    x =[LpVariable('x'+str(j), lowBound=0, cat='Binary') for j in range(n)]
    
    primal += lpSum(x[j] for j in range(n)) # 目的関数の設定
    for i in range(m): # 制約条件の付加
        primal += lpDot(A[i], x) >= 1, 'ineq'+str(i)

    # 最適値が1より大きい場合は近似解となる
    primal.solve()
    if value(primal.objective) - value(dual.objective) < 1.0-MEPS:
        print('Optimal solution found: ')
    else:
        print('Approximated solution found: ')
    
    # 最終的に1となった物のみ抽出
    K = [j for j in range(n) if x[j].varValue > MEPS]
    print("K:", K)
    print("A[:,K]\n", A[:,K]) # この行列においてかぶっているものはどちらに入れてもよい
    
    # 表示
    result = []
    itms = set(range(m))
    for j in K:
        J = {i for i in range(m) if A[i,j] > MEPS and i in itms}
        r = [w[i] for i in J]
        itms -= J
        result.append(r)
    print(result)

In [4]:
# 入れ物の容量を定義
capacity =25

# item番号を付与
items = set(range(10))
np.random.seed(1)

# ランダムにitemの量を定義
w ={i:np.random.randint(5,10) for i in items}
w2 = [w[i] for i in items]
print(w2)

# 計算
binpacking(capacity, w)

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9]
列生成：0 の計算を行います...
costs: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0}
weights: {0: 8, 1: 9, 2: 5, 3: 6, 4: 8, 5: 5, 6: 5, 7: 6, 8: 9, 9: 9}
val:  4.0
sol:  {0: 0, 1: 0, 2: 1, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0}
A:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
列生成：1 の計算を行います...
costs: {0: 1.0, 1: 1.0, 2: 0.0, 3: 0.0, 4: 1.0, 5: 0.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0}
weights: {0: 8, 1: 9, 2: 5, 3: 6, 4: 8, 5: 5, 6: 5, 7: 6, 8: 9, 9: 9}
val:  3.0
sol:  {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 0, 9: 0}
A:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.

  0.]]
列生成：15 の計算を行います...
costs: {0: 0.16666667, 1: 0.5, 2: 0.33333333, 3: 0.16666667, 4: 0.5, 5: 0.33333333, 6: 0.16666667, 7: 0.16666667, 8: 0.33333333, 9: 0.33333333}
weights: {0: 8, 1: 9, 2: 5, 3: 6, 4: 8, 5: 5, 6: 5, 7: 6, 8: 9, 9: 9}
val:  1.3333333299999999
sol:  {0: 0, 1: 1, 2: 1, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
A:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.
  1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0.
  0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
  1. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
  0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.
  1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1.
  0. 0.]


列生成：23 の計算を行います...
costs: {0: 0.28571429, 1: 0.35714286, 2: 0.21428571, 3: 0.25, 4: 0.35714286, 5: 0.17857143, 6: 0.17857143, 7: 0.25, 8: 0.39285714, 9: 0.39285714}
weights: {0: 8, 1: 9, 2: 5, 3: 6, 4: 8, 5: 5, 6: 5, 7: 6, 8: 9, 9: 9}
val:  1.07142857
sol:  {0: 0, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0}
A:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.
  1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0.
  0. 1. 1. 1. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.
  1. 1. 0. 1. 1. 1. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  1. 0. 0. 1. 0. 1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
  0. 1. 1. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
  0. 0. 0. 1. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 

Optimal solution found: 
K: [26, 36, 37]
A[:,K]
 [[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
[[8, 9, 8], [9, 5, 5, 6], [9, 6, 5]]


# ナップサック問題は以下のようにPULPを使って解いても良い

In [5]:
def KPS(capacity, items, costs, weights):
    knapsack = LpProblem(name='knapsack', sense=LpMaximize)
    x ={j: LpVariable('x'+str(j), lowBound=0, cat='Binary') for j in items}
    
    knapsack += lpSum(costs[j]*x[j] for j in items) # 目的関数の設定
    knapsack += lpSum(weights[j]*x[j] for j in items) <= capacity, 'weights'

    knapsack.solve()
    xx= {j: int(x[j].varValue) for j in items}
    return LpStatus[knapsack.status], value(knapsack.objective), xx

# 以下のように定式化すると小規模であれば解くことが可能

In [6]:
from pulp import *
MEPS = 1.0e-8

def binpacking2(capacity, w):
    n = len(w)
    items = range(n)
    bpprob =  LpProblem(name='BinPacking2', sense=LpMinimize)
    z = [LpVariable('z'+str(j), lowBound=0,cat='Binary') for j in items] #j容器に何か入っているかどうかを表す
    x =[[LpVariable('x'+str(i)+str(j), lowBound=0, cat='Binary') for j in items] for i in items] # 品物iがj容器に入っているかどうかを表す
    
    # 容器の数を最小化
    bpprob += lpSum(z[i] for i in items)
    # 品物iはどこかに入る
    for i in items:
        bpprob += lpSum(x[i][j] for j in items) == 1
    # 容器j内の品物は必ず容量以下になる
    for j in items:
        bpprob += lpSum(x[i][j]*w[i] for i in items) <= capacity*z[j]

    # 解く
    bpprob.solve()
    result = []
    for j in items:
        if z[j].varValue > MEPS:
            r = [w[i] for i in items if x[i][j].varValue > MEPS]
            result.append(r)
    print(result)

In [7]:
capacity =25
items = set(range(10))
np.random.seed(1)
w ={i:np.random.randint(5,10) for i in items}
w2 = [w[i] for i in items]
print(w2)

binpacking2(capacity, w)

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9]
[[9, 6, 5, 5], [5, 6, 9], [8, 8, 9]]
